# Read Data from LakeHouse using trino

In [3]:
import trino
import pandas as pd
# Create a connection
conn = trino.dbapi.connect(
    host="localhost",        # Trino coordinator host
    port=8086,               # Default Trino port
    user="anoir",            # Your username
    catalog="iceberg",       # Catalog (e.g., iceberg, hive, jdbc)
    schema="gold_layer"         # Schema inside the catalog
)

# Create a cursor
cur = conn.cursor()

In [11]:
# Execute a query
cur.execute("SELECT * FROM dim_movies LIMIT 10")

# Fetch all rows
rows = cur.fetchall()

# Get column names from cursor.description
columns = [col[0] for col in cur.description]

# Convert to DataFrame
df = pd.DataFrame(rows, columns=columns)

df.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
from sqlalchemy.engine import create_engine
import pandas as pd

engine = create_engine("trino://anoir@localhost:8086/iceberg/gold_layer")

# Pull query results straight into a DataFrame
movies = pd.read_sql("SELECT * FROM dim_movies", engine)
users = pd.read_sql("SELECT * FROM dim_users", engine)
ratings = pd.read_sql("SELECT * FROM fact_ratings", engine)


print(ratings.tail())

         user_id  movie_id  rating  timestamp
1000204      445      2011       3  976239845
1000205      445      3751       5  976240768
1000206      445      2012       4  976239944
1000207      445      1210       5  976239747
1000208      445      3752       2  976241782


# Prepare the Data

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [25]:
# Merge into one training set
df = ratings.merge(users, on="user_id").merge(movies, on="movie_id")

In [26]:
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip_code,title,genres
0,1228,1111,4,975353839,M,25,1,94720,Microcosmos (Microcosmos: Le peuple de l'herbe...,Documentary
1,1228,2858,4,974837243,M,25,1,94720,American Beauty (1999),Comedy|Drama
2,1228,446,4,975354355,M,25,1,94720,Farewell My Concubine (1993),Drama|Romance
3,1228,3809,2,990901447,M,25,1,94720,What About Bob? (1991),Comedy
4,1228,457,4,975353574,M,25,1,94720,"Fugitive, The (1993)",Action|Thriller


In [27]:
df["genres_list"] = df["genres"].str.split("|")

In [28]:
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip_code,title,genres,genres_list
0,1228,1111,4,975353839,M,25,1,94720,Microcosmos (Microcosmos: Le peuple de l'herbe...,Documentary,[Documentary]
1,1228,2858,4,974837243,M,25,1,94720,American Beauty (1999),Comedy|Drama,"[Comedy, Drama]"
2,1228,446,4,975354355,M,25,1,94720,Farewell My Concubine (1993),Drama|Romance,"[Drama, Romance]"
3,1228,3809,2,990901447,M,25,1,94720,What About Bob? (1991),Comedy,[Comedy]
4,1228,457,4,975353574,M,25,1,94720,"Fugitive, The (1993)",Action|Thriller,"[Action, Thriller]"


In [29]:
# Multi-hot encode
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genre_features = mlb.fit_transform(df["genres_list"])

# Create a DataFrame with genre columns
genres_df = pd.DataFrame(genre_features, columns=mlb.classes_, index=df.index)

In [30]:
# Merge back
df = pd.concat([df, genres_df], axis=1)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip_code,title,genres,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1228,1111,4,975353839,M,25,1,94720,Microcosmos (Microcosmos: Le peuple de l'herbe...,Documentary,...,0,0,0,0,0,0,0,0,0,0
1,1228,2858,4,974837243,M,25,1,94720,American Beauty (1999),Comedy|Drama,...,0,0,0,0,0,0,0,0,0,0
2,1228,446,4,975354355,M,25,1,94720,Farewell My Concubine (1993),Drama|Romance,...,0,0,0,0,0,1,0,0,0,0
3,1228,3809,2,990901447,M,25,1,94720,What About Bob? (1991),Comedy,...,0,0,0,0,0,0,0,0,0,0
4,1228,457,4,975353574,M,25,1,94720,"Fugitive, The (1993)",Action|Thriller,...,0,0,0,0,0,0,0,1,0,0


In [35]:
df = df.drop(["genres", "genres_list"], axis=1)

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'gender', 'age',
       'occupation', 'zip_code', 'title', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [38]:
df["label"] = (df["rating"] >= 4).astype(int)

In [39]:
# Features: demographics + genres
feature_cols = ["gender", "age", "occupation"] + list(mlb.classes_)
X = df[feature_cols]
y = df["label"]

,gender,age,occupation,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,M,25,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,M,25,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,M,25,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,25,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Build the model

In [44]:
categorical = ["gender", "occupation"]
numeric = ["age"]
genre_cols = list(mlb.classes_)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", "passthrough", numeric),
        ("genres", "passthrough", genre_cols)
    ]
)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", GradientBoostingClassifier(n_estimators=200, random_state=42))
])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.25      0.35     84974
           1       0.61      0.87      0.72    115068

    accuracy                           0.61    200042
   macro avg       0.60      0.56      0.54    200042
weighted avg       0.60      0.61      0.56    200042



In [56]:
def recommend_for_user(user_id, top_n=10):
    user_row = users.loc[users["user_id"] == user_id].iloc[0]
    rated_movies = ratings.loc[ratings["user_id"] == user_id, "movie_id"].unique()
    candidates = movies[~movies["movie_id"].isin(rated_movies)].copy()

    # Add user features
    candidates["gender"] = user_row["gender"]
    candidates["age"] = user_row["age"]
    candidates["occupation"] = user_row["occupation"]

    # Ensure genre one-hot columns exist
    for g in mlb.classes_:
        if g not in candidates.columns:
            candidates[g] = 0

    X_cand = candidates[["gender", "age", "occupation"] + list(mlb.classes_)]
    scores = clf.predict_proba(X_cand)[:, 1]
    candidates["score"] = scores

    return candidates.sort_values("score", ascending=False).head(top_n)[["movie_id", "title", "score"]]


# Example usage:
recs = recommend_for_user(user_id=777, top_n=10)
print(recs)

    movie_id                     title    score
2          3   Grumpier Old Men (1995)  0.48501
3          4  Waiting to Exhale (1995)  0.48501
5          6               Heat (1995)  0.48501
7          8       Tom and Huck (1995)  0.48501
8          9       Sudden Death (1995)  0.48501
9         10          GoldenEye (1995)  0.48501
12        13              Balto (1995)  0.48501
13        14              Nixon (1995)  0.48501
14        15   Cutthroat Island (1995)  0.48501
15        16             Casino (1995)  0.48501


In [52]:
import joblib

# Save the trained pipeline
joblib.dump(clf, "recommendation_model.pkl")

# Later: load it back
clf_loaded = joblib.load("recommendation_model.pkl")

# Use it exactly like before
y_pred = clf_loaded.predict(X_test)


In [53]:
y_pred

array([1, 1, 1, ..., 0, 1, 1], shape=(200042,))